In [ ]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.colors
import os
import glob
import logging
from pathlib import Path
from typing import Optional, List, Dict
from collections import deque
import gc # Import the garbage collector module

# --- Configuration ---
PLOTLY_TEMPLATE = "plotly_dark"
BASE_DATA_DIR = os.path.join("backtest", "data")
RISK_FREE_RATE = 0.02  # Annual risk-free rate for Sharpe/Sortino calculations

# Configure logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# --- Helper Functions ---

def find_latest_backtest_dir(base_dir: str) -> Optional[str]:
    """Finds the most recently created backtest directory."""
    try:
        list_of_dirs = [d for d in Path(base_dir).iterdir() if d.is_dir() and '_backtest_' in d.name]
        if not list_of_dirs:
            logging.warning(f"No directories matching '*_backtest_*' found in {base_dir}")
            return None
        return str(max(list_of_dirs, key=lambda d: d.stat().st_mtime))
    except FileNotFoundError:
        logging.error(f"Base data directory not found: {base_dir}")
        return None
    except Exception as e:
        logging.error(f"Error finding latest backtest directory: {e}", exc_info=True)
        return None

def load_csv(file_path: str, index_col=None) -> Optional[pd.DataFrame]:
    """Loads a CSV file into a pandas DataFrame with robust error handling."""
    path_obj = Path(file_path)
    if not path_obj.is_file():
        logging.warning(f"CSV file not found: {file_path}")
        return None
    if path_obj.stat().st_size == 0:
        logging.warning(f"CSV file is empty: {file_path}")
        return pd.DataFrame()
    try:
        df = pd.read_csv(file_path, index_col=index_col)
        if 'timestamp' in df.columns:
            df['timestamp'] = pd.to_datetime(df['timestamp'], errors='coerce', utc=True).dt.tz_convert('America/New_York')
        elif df.index.name == 'timestamp':
            df.index = pd.to_datetime(df.index, errors='coerce', utc=True).tz_convert('America/New_York')
        return df
    except Exception as e:
        logging.error(f"Error loading CSV file {file_path}: {e}", exc_info=True)
        return None

# --- Trade Cycle Processing ---
def process_trades_into_cycles(df_trades: Optional[pd.DataFrame]) -> Optional[pd.DataFrame]:
    """Processes a raw trade log into distinct trade cycles (entry to exit) using FIFO."""
    if df_trades is None or df_trades.empty:
        return None
    df_trades = df_trades.sort_values(by=['ticker', 'timestamp']).reset_index(drop=True)
    completed_cycles, open_positions = [], {}
    for _, trade in df_trades.iterrows():
        ticker, signal, shares, price, timestamp = trade['ticker'], trade['signal_type'], trade['shares'], trade['fill_price'], trade['timestamp']
        if ticker not in open_positions: open_positions[ticker] = deque()
        if signal == 'BUY':
            open_positions[ticker].append({'shares': shares, 'price': price, 'timestamp': timestamp})
        elif signal == 'SELL':
            sell_shares = shares
            while sell_shares > 0 and open_positions[ticker]:
                buy_trade = open_positions[ticker][0]
                matched_shares = min(sell_shares, buy_trade['shares'])
                completed_cycles.append({
                    'ticker': ticker, 'entry_time': buy_trade['timestamp'], 'exit_time': timestamp,
                    'duration_hours': (timestamp - buy_trade['timestamp']).total_seconds() / 3600,
                    'pnl': (price - buy_trade['price']) * matched_shares, 'entry_price': buy_trade['price'],
                    'exit_price': price, 'shares': matched_shares
                })
                sell_shares -= matched_shares
                buy_trade['shares'] -= matched_shares
                if buy_trade['shares'] < 1e-6: open_positions[ticker].popleft()
    if not completed_cycles: return None
    df_cycles = pd.DataFrame(completed_cycles)
    df_cycles['return_pct'] = (df_cycles['exit_price'] - df_cycles['entry_price']) / df_cycles['entry_price']
    return df_cycles

# --- Metrics Calculation Engine ---
def calculate_all_metrics(df_abs: pd.DataFrame, df_trades: Optional[pd.DataFrame] = None, df_cycles: Optional[pd.DataFrame] = None, risk_free_rate: float = RISK_FREE_RATE) -> Dict[str, any]:
    """
    Calculates a comprehensive set of annualized performance and risk metrics,
    with an emphasis on accurate, trade-based statistics.
    """
    metrics = {}
    if df_abs is None or 'portfolio_value' not in df_abs.columns or df_abs.empty:
        return {}
    
    equity_curve = df_abs.set_index('timestamp')['portfolio_value'].dropna()
    if equity_curve.empty:
        return {}

    # --- General Backtest Information ---
    initial_capital, final_equity = equity_curve.iloc[0], equity_curve.iloc[-1]
    metrics['Start Date'] = equity_curve.index[0]
    metrics['End Date'] = equity_curve.index[-1]
    metrics['Initial Capital'] = initial_capital
    metrics['Final Capital'] = final_equity

    # --- Equity Curve-Based Performance Metrics ---
    daily_returns = equity_curve.resample('D').last().pct_change().dropna()
    net_profit = final_equity - initial_capital
    total_days = (metrics['End Date'] - metrics['Start Date']).days
    
    metrics['Net Profit'] = net_profit
    metrics['Cumulative Returns'] = (final_equity / initial_capital) - 1 if initial_capital > 0 else 0
    metrics['Annualised Return'] = (1 + metrics['Cumulative Returns']) ** (365.25 / max(total_days, 1)) - 1

    metrics['Return on Investment (ROI)'] = net_profit / initial_capital if initial_capital > 0 else 0
    drawdown = (equity_curve - equity_curve.cummax()) / equity_curve.cummax()
    metrics['Maximum Drawdown'] = drawdown.min()
    
    annual_volatility = daily_returns.std() * np.sqrt(252)
    metrics['Volatility (Annualised)'] = annual_volatility
    
    if annual_volatility > 0:
        daily_rf = (1 + risk_free_rate)**(1/252) - 1
        excess_returns = daily_returns - daily_rf
        metrics['Sharpe Ratio (Annualised)'] = (excess_returns.mean() / excess_returns.std()) * np.sqrt(252) if excess_returns.std() > 0 else 0
        downside_std = excess_returns[excess_returns < 0].std()
        metrics['Sortino Ratio (Annualised)'] = (excess_returns.mean() / downside_std) * np.sqrt(252) if downside_std > 0 else np.inf
        metrics['Calmar Ratio'] = metrics['Annualised Return'] / abs(metrics['Maximum Drawdown']) if metrics['Maximum Drawdown'] < 0 else np.inf
    else:
        metrics.update({'Sharpe Ratio (Annualised)': 0, 'Sortino Ratio (Annualised)': 0, 'Calmar Ratio': 0})

    # --- ACCURATE TRADE-BASED METRICS ---
    # Use pre-computed cycles if available, otherwise compute them
    if df_cycles is None and df_trades is not None:
        df_cycles = process_trades_into_cycles(df_trades)

    if df_cycles is not None and not df_cycles.empty:
        winning_trades_df = df_cycles[df_cycles['pnl'] > 0]
        losing_trades_df = df_cycles[df_cycles['pnl'] < 0]
        total_closed_trades = len(df_cycles)
        
        # Win/Loss counts and rates
        metrics['Total Closed Trades'] = total_closed_trades
        metrics['Number of Winning Trades'] = len(winning_trades_df)
        metrics['Number of Losing Trades'] = len(losing_trades_df)
        metrics['Win Rate (% of Trades)'] = len(winning_trades_df) / total_closed_trades if total_closed_trades > 0 else 0
        
        # PnL-based ratios
        gross_profit = winning_trades_df['pnl'].sum()
        gross_loss = abs(losing_trades_df['pnl'].sum())
        metrics['Profit Factor'] = gross_profit / gross_loss if gross_loss > 0 else np.inf

        avg_win_pnl = winning_trades_df['pnl'].mean()
        avg_loss_pnl = abs(losing_trades_df['pnl'].mean())
        metrics['Average Winning Trade ($)'] = avg_win_pnl
        metrics['Average Losing Trade ($)'] = avg_loss_pnl
        metrics['Payoff Ratio'] = avg_win_pnl / avg_loss_pnl if avg_loss_pnl > 0 else np.inf

    if df_trades is not None:
        metrics['Total Orders'] = len(df_trades) # Total buy/sell orders placed
        
    return metrics

# --- Visualization Suite ---

def display_metrics_table(metrics: dict, title="Backtest Performance Metrics"):
    """Displays the calculated metrics in a clean, professional table with improved categorization."""
    # REVISED: Re-categorized metrics for better readability
    categories = {
        'Backtest Summary': [
            'Start Date', 'End Date', 'Initial Capital', 'Final Capital'
        ],
        'Overall Performance': [
            'Cumulative Returns', 'Annualised Return', 'Net Profit', 'Return on Investment (ROI)'
        ],
        'Risk Metrics (Annualised)': [
            'Maximum Drawdown', 'Volatility (Annualised)', 'Sharpe Ratio (Annualised)', 'Sortino Ratio (Annualised)', 'Calmar Ratio'
        ],
        'Trade Statistics': [
            'Total Orders', 'Total Closed Trades', 'Number of Winning Trades', 'Number of Losing Trades',
            'Win Rate (% of Trades)', 'Profit Factor', 'Payoff Ratio', 'Average Winning Trade ($)', 'Average Losing Trade ($)'
        ]
    }
    
    header, cells = ['Category', 'Metric', 'Value'], [[], [], []]
    
    for cat, names in categories.items():
        is_first_in_cat = True
        for name in names:
            if name in metrics and pd.notna(metrics[name]):
                cells[0].append(f"<b>{cat}</b>" if is_first_in_cat else "")
                cells[1].append(name)
                val = metrics[name]
                is_first_in_cat = False
                
                # REVISED: More specific formatting logic for each data type
                if isinstance(val, (pd.Timestamp, pd.Timestamp)):
                    fmt = val.strftime('%Y-%m-%d')
                elif name in ['Initial Capital', 'Final Capital', 'Net Profit', 'Average Winning Trade ($)', 'Average Losing Trade ($)']:
                    fmt = f"${val:,.2f}"
                elif any(s in name for s in ['Return', 'Rate', 'Drawdown', 'Volatility', 'ROI']):
                    fmt = f"{val:.2%}"
                elif any(s in name for s in ['Ratio', 'Factor']):
                    fmt = f"{val:.2f}"
                else: # Default for integers like counts
                    fmt = f"{val:,.0f}"

                cells[2].append(fmt)

    fig = go.Figure(data=[go.Table(
        header=dict(
            values=[f"<b>{h}</b>" for h in header],
            fill_color='#2c3e50',
            align='left',
            font=dict(color='white', size=14)
        ),
        cells=dict(
            values=cells,
            fill_color='#34495e',
            align='left',
            font=dict(color='white', size=12),
            height=30
        )
    )])
    
    # REVISED: Increased height to accommodate all new metrics
    fig.update_layout(
        title_text=f"<b>{title}</b>",
        title_x=0.5,
        template=PLOTLY_TEMPLATE,
        height=850  # Increased height
    )
    fig.show()

def plot_performance_and_drawdown(df_abs: pd.DataFrame, df_trades: Optional[pd.DataFrame], df_benchmark: Optional[pd.DataFrame], title_suffix=""):
    """Plots equity curve, benchmark, trades, and drawdown."""
    if df_abs is None or 'portfolio_value' not in df_abs.columns: return
    fig = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.05, row_heights=[0.7, 0.3])
    equity_curve = df_abs.set_index('timestamp')['portfolio_value']
    fig.add_trace(go.Scatter(x=equity_curve.index, y=equity_curve, mode='lines', name='Strategy Value ($)', line=dict(color='#3498db', width=2.5)), row=1, col=1)
    if df_benchmark is not None and not df_benchmark.empty:
        fig.add_trace(go.Scatter(x=df_benchmark['timestamp'], y=df_benchmark['buy_and_hold_value'], mode='lines', name='Buy & Hold Benchmark ($)', line=dict(color='#f1c40f', width=1.5, dash='dash')), row=1, col=1)
    if df_trades is not None and not df_trades.empty:
        buys, sells = df_trades[df_trades['signal_type'] == 'BUY'], df_trades[df_trades['signal_type'] == 'SELL']
        fig.add_trace(go.Scatter(x=buys['timestamp'], y=equity_curve.reindex(buys['timestamp'], method='pad'), mode='markers', name='Buy', marker=dict(color='#2ecc71', size=8, symbol='triangle-up')), row=1, col=1)
        fig.add_trace(go.Scatter(x=sells['timestamp'], y=equity_curve.reindex(sells['timestamp'], method='pad'), mode='markers', name='Sell', marker=dict(color='#e74c3c', size=8, symbol='triangle-down')), row=1, col=1)
    drawdown = (equity_curve - equity_curve.cummax()) / equity_curve.cummax()
    fig.add_trace(go.Scatter(x=drawdown.index, y=drawdown, fill='tozeroy', mode='lines', name='Drawdown', line=dict(color='#e74c3c', width=1)), row=2, col=1)
    fig.update_layout(title_text=f"<b>Performance vs. Benchmark and Drawdown{title_suffix}</b>", title_x=0.5, template=PLOTLY_TEMPLATE, legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1), hovermode="x unified", height=700)
    fig.update_yaxes(title_text="Portfolio Value ($)", row=1, col=1); fig.update_yaxes(title_text="Drawdown", row=2, col=1, tickformat=".2%")
    fig.show()

def plot_stacked_portfolio_composition(df_minute: pd.DataFrame, title_suffix=""):
    """CORRECTED: Plots minute-by-minute portfolio composition including cash."""
    if df_minute is None or df_minute.empty:
        logging.warning("Minute performance data is empty. Skipping composition plot.")
        return
    df_plot = df_minute.copy().set_index('timestamp')
    plot_cols = [col for col in df_plot.columns if col.endswith('_value')]
    fig = go.Figure()
    colors = plotly.colors.qualitative.Plotly
    for i, col in enumerate(plot_cols):
        color = '#B0B0B0' if col == 'cash_value' else colors[i % len(colors)]
        fig.add_trace(go.Scatter(
            x=df_plot.index, y=df_plot[col], mode='lines',
            name=col.replace('_', ' ').title(),
            stackgroup='one', line=dict(width=0.5, color=color),
            hovertemplate=f"<b>{col.replace('_', ' ').title()}</b><br>Value: $%{{y:,.2f}}<br>Date: %{{x}}<extra></extra>"
        ))
    fig.update_layout(
        title_text=f"<b>Portfolio Composition Over Time (Minute by Minute){title_suffix}</b>", title_x=0.5,
        xaxis_title="Timestamp", yaxis_title="Notional Value ($)",
        template=PLOTLY_TEMPLATE, hovermode="x unified", legend_title_text='Components'
    )
    fig.show()

def plot_rolling_volatility(results_dir: str, df_risk_comp: pd.DataFrame, title_suffix=""):
    """ENHANCED: Plots rolling volatility for the portfolio AND individual tickers."""
    all_rolling_files = glob.glob(os.path.join(results_dir, "*D_Rolling.csv"))
    if not all_rolling_files: return
    fig = go.Figure()
    colors = plotly.colors.qualitative.Vivid
    df_rolling = load_csv(sorted(all_rolling_files)[0])
    if df_rolling is not None and not df_rolling.empty:
        window = os.path.basename(sorted(all_rolling_files)[0]).split('_Rolling.csv')[0]
        portfolio_vol_col = f'portfolio_pct_ret_vol_{window.lower()}'
        if portfolio_vol_col in df_rolling.columns:
            fig.add_trace(go.Scatter(
                x=df_rolling['timestamp'], y=df_rolling[portfolio_vol_col], mode='lines',
                name=f'Total Portfolio Volatility ({window})',
                line=dict(color='#3498db', width=2.5)
            ))
    if df_risk_comp is not None and not df_risk_comp.empty:
        tickers = df_risk_comp['ticker'].unique()
        for i, ticker in enumerate(tickers):
            for file_path in sorted(all_rolling_files):
                df_rolling_ticker = load_csv(file_path)
                if df_rolling_ticker is None: continue
                window = os.path.basename(file_path).split('_Rolling.csv')[0]
                vol_col_ticker = f'{ticker}_pct_ret_vol_{window.lower()}'
                if vol_col_ticker in df_rolling_ticker.columns:
                     fig.add_trace(go.Scatter(
                        x=df_rolling_ticker['timestamp'], y=df_rolling_ticker[vol_col_ticker], mode='lines',
                        name=f'{ticker} Volatility ({window})',
                        line=dict(color=colors[i % len(colors)], width=1.5, dash='dash'),
                        visible='legendonly'
                    ))
    fig.update_layout(
        title_text=f"<b>Rolling Annualized Volatility{title_suffix}</b>", title_x=0.5,
        yaxis_title="Annualized Volatility (Std. Dev.)", xaxis_title="Timestamp",
        template=PLOTLY_TEMPLATE, hovermode="x unified",
        legend_title_text='Series (Click to Toggle)'
    )
    fig.update_yaxes(tickformat=".2%"); fig.show()

def plot_pnl_distribution(df_cycles: Optional[pd.DataFrame], title_suffix=""):
    """Plots the distribution of Profit and Loss (PnL) per trade cycle."""
    if df_cycles is None or df_cycles.empty: return
    wins, losses = df_cycles[df_cycles['pnl'] >= 0], df_cycles[df_cycles['pnl'] < 0]
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=wins['pnl'], name='Winning Trades', marker_color='#2ecc71', nbinsx=50))
    fig.add_trace(go.Histogram(x=losses['pnl'], name='Losing Trades', marker_color='#e74c3c', nbinsx=50))
    fig.update_layout(
        title_text=f"<b>Distribution of Profit & Loss per Trade{title_suffix}</b>", title_x=0.5,
        xaxis_title="Profit/Loss ($)", yaxis_title="Number of Trades",
        template=PLOTLY_TEMPLATE, barmode='overlay'
    )
    fig.update_traces(opacity=0.75); fig.show()

def plot_risk_components(df_risk_comp: pd.DataFrame, df_cov_matrix: pd.DataFrame, df_cycles: Optional[pd.DataFrame], title_suffix=""):
    """
    CORRECTED & ENHANCED: Plots Ticker PnL, Weights, Volatility, and the Correlation Matrix.
    This version separates PnL into its own subplot to prevent overlapping bars and improve clarity.
    """
    if df_risk_comp is None or df_risk_comp.empty or df_cov_matrix is None or df_cov_matrix.empty:
        return

    # The variable `df_cov_matrix` is a misnomer from the calling code in `main`,
    # as it loads 'annualized_correlation_matrix.csv'. It IS the correlation matrix.
    # We now use the matrix directly.
    df_corr_matrix = df_cov_matrix

    if df_cycles is not None and not df_cycles.empty:
        pnl_per_ticker = df_cycles.groupby('ticker')['pnl'].sum().rename('Total PnL')
        df_plot_data = df_risk_comp.set_index('ticker').join(pnl_per_ticker).fillna(0).reset_index()
    else:
        df_plot_data = df_risk_comp.copy()
        df_plot_data['Total PnL'] = 0

    # --- MODIFICATION: Changed subplot layout to 2 rows to separate PnL chart ---
    fig = make_subplots(
        rows=2, cols=2,
        column_widths=[0.45, 0.55],
        row_heights=[0.6, 0.4],
        specs=[
            [{}, {"rowspan": 2}],
            [{}, None]
        ],
        subplot_titles=(
            "<b>Ticker Weight & Volatility</b>",
            "<b>Annualized Correlation Matrix</b>",
            "<b>Ticker PnL ($)</b>"
        ),
        vertical_spacing=0.15
    )
    
    # --- Top-Left Plot: Ticker Weight and Volatility Bars ---
    fig.add_trace(go.Bar(x=df_plot_data['ticker'], y=df_plot_data['weight'], name='Weight', marker_color='#3498db'), row=1, col=1)
    fig.add_trace(go.Bar(x=df_plot_data['ticker'], y=df_plot_data['annualized_volatility'], name='Volatility', marker_color='#f1c40f'), row=1, col=1)
    
    # --- Bottom-Left Plot: Total PnL Bars in a separate subplot ---
    fig.add_trace(go.Bar(x=df_plot_data['ticker'], y=df_plot_data['Total PnL'], name='Total PnL ($)', marker_color='#2ecc71'), row=2, col=1)
    
    # --- Right Plot: Heatmap (Spanning both rows) ---
    fig.add_trace(go.Heatmap(
        z=df_corr_matrix.values,
        x=df_corr_matrix.columns,
        y=df_corr_matrix.index,
        colorscale='RdBu',
        zmid=0,
        text=np.around(df_corr_matrix.values, 2),
        texttemplate="%{text}",
        hoverongaps=False
    ), row=1, col=2)
    
    # --- Layout Updates for the new structure ---
    fig.update_layout(
        title_text=f"<b>Static Portfolio Risk & Performance Analysis{title_suffix}</b>", title_x=0.5,
        template=PLOTLY_TEMPLATE,
        barmode='group', # This will now work correctly for each subplot
        legend=dict(orientation="h", yanchor="bottom", y=1.05, xanchor="right", x=1),
        height=750 # Adjusted height for the new layout
    )
    
    # Update axes for each specific subplot
    fig.update_yaxes(title_text="Weight / Volatility", tickformat=".2%", row=1, col=1)
    fig.update_yaxes(title_text="Total PnL ($)", row=2, col=1)
    fig.update_yaxes(autorange='reversed', row=1, col=2) # For the heatmap
    fig.show()


def plot_rolling_portfolio_risk(df_rolling_risk: pd.DataFrame, title_suffix=""):
    """Plots the 30-day rolling annualized portfolio risk."""
    if df_rolling_risk is None or df_rolling_risk.empty: return
    risk_col = [col for col in df_rolling_risk.columns if 'portfolio_risk' in col][0]
    fig = go.Figure(data=[go.Scatter(x=df_rolling_risk['timestamp'], y=df_rolling_risk[risk_col], mode='lines', name='Portfolio Risk', line=dict(color='#e74c3c', width=2))])
    fig.update_layout(
        title_text=f"<b>Rolling Ann. Portfolio Risk (30-Day){title_suffix}</b>", title_x=0.5,
        yaxis_title="Annualized Volatility (Std. Dev.)", xaxis_title="Timestamp",
        template=PLOTLY_TEMPLATE, yaxis_tickformat=".2%",
        xaxis=dict(
            rangebreaks=[
                dict(bounds=["sat", "mon"], pattern="day of week")
            ]
        )
    )
    fig.show()

# --- Main Execution Block ---

if __name__ == "__main__":
    logging.info("--- Starting Comprehensive Backtest Visualization Script ---")
    target_dir = find_latest_backtest_dir(BASE_DATA_DIR)
    if not target_dir:
        logging.error("No valid backtest directory found. Exiting.")
    else:
        logging.info(f"Visualizing results from: {target_dir}")
        portfolio_id = os.path.basename(target_dir).split('_backtest_')[-1]
        title_suffix = f" (Portfolio: {portfolio_id})"
        df_cycles = None # Initialize to ensure it's in scope

        # --- Section 1: Core Performance (Metrics, Equity Curve, PnL) ---
        logging.info("Generating core performance visuals (Metrics, Equity, Drawdown, PnL)...")
        df_abs = load_csv(os.path.join(target_dir, "performance_timeseries_absolute.csv"))
        df_trades = load_csv(os.path.join(target_dir, "trade_log.csv"))
        
        if df_abs is not None:
            # Pre-compute cycles once as it's needed for multiple plots
            df_cycles = process_trades_into_cycles(df_trades)

            # Generate and display metrics table
            metrics = calculate_all_metrics(df_abs, df_trades=df_trades, df_cycles=df_cycles)
            display_metrics_table(metrics, title=f"Comprehensive Performance Metrics{title_suffix}")
            del metrics # No longer needed

            # Generate Performance vs. Benchmark and Drawdown plot
            df_benchmark = load_csv(os.path.join(target_dir, "benchmark_buy_and_hold_performance.csv"))
            plot_performance_and_drawdown(df_abs, df_trades, df_benchmark, title_suffix)
            
            # Generate PnL distribution plot
            plot_pnl_distribution(df_cycles, title_suffix)

            # Clean up data for this section
            del df_abs, df_trades, df_benchmark
            gc.collect()
            logging.info("Core performance visuals generated. Memory cleared.")
        else:
            logging.error("Could not load essential absolute performance data. Some plots will be skipped.")
            # Still try to compute cycles for other plots if trades exist
            if df_trades is not None:
                df_cycles = process_trades_into_cycles(df_trades)
                del df_trades
                gc.collect()

        # --- Section 2: Portfolio Composition ---
        logging.info("Generating portfolio composition visual...")
        df_minute_perf = load_csv(os.path.join(target_dir, "performance_timeseries_minute_by_minute.csv"))
        if df_minute_perf is not None:
            plot_stacked_portfolio_composition(df_minute_perf, title_suffix)
            del df_minute_perf
            gc.collect()
            logging.info("Portfolio composition visual generated. Memory cleared.")
        else:
            logging.warning("Minute-by-minute performance data not found. Skipping composition plot.")

        # --- Section 3: Risk Analysis ---
        logging.info("Generating risk analysis visuals (Static Components, Rolling Volatility)...")
        df_risk_comp = load_csv(os.path.join(target_dir, "portfolio_risk_components.csv"))
        df_cov_matrix = load_csv(os.path.join(target_dir, "annualized_correlation_matrix.csv"), index_col=0)

        # Plot static risk components and correlation
        plot_risk_components(df_risk_comp, df_cov_matrix, df_cycles, title_suffix)
        del df_cov_matrix, df_cycles # Clean up cycles now, it's no longer needed
        gc.collect()

        # Plot rolling volatility for portfolio and individual tickers
        plot_rolling_volatility(target_dir, df_risk_comp, title_suffix)
        if df_risk_comp is not None: del df_risk_comp
        gc.collect()
        logging.info("Static and rolling risk visuals generated. Memory cleared.")

        # --- Section 4: Rolling Portfolio Risk ---
        logging.info("Generating rolling portfolio risk visual...")
        df_rolling_risk = load_csv(os.path.join(target_dir, "rolling_portfolio_risk.csv"))
        if df_rolling_risk is not None:
            plot_rolling_portfolio_risk(df_rolling_risk, title_suffix)
            del df_rolling_risk
            gc.collect()
            logging.info("Rolling portfolio risk visual generated. Memory cleared.")
        else:
            logging.warning("Rolling portfolio risk data not found. Skipping plot.")




    logging.info("--- Visualization Script Finished ---")